# Transportation model under varying vehicle capacity

## Problem Description
**Transportation Problem with Varying Vehicular Capacities**

**Objective:**

Minimize the transportation cost of goods from suppliers(Dangote, Elephant) to consumers(Intra, SCC, Julius Berger) using different types of vehicles (Truck 1, Truck 2), each with its own capacity.

**Data:**

- Suppliers: Dangote, Elephant
- Consumers: Intra, SCC, Julius Berger
- Vehicle Types: Truck 1, Truck 2

**Supply and Demand:**

- Supply from Dangote: 700 units
- Supply from Elephant: 800 units
- Demand at Intra: 550 units
- Demand at SCC: 500 units
- Demand at Julius Berger: 450 units

**Transportation Costs (per unit):**

- Dangote to Intra using Truck 1: N400
- Dangote to Intra using Truck 2: N350
- Dangote to SCC using Truck 1: N350
- Dangote to SCC using Truck 2: N350
- Dangote to Julius Berger using Truck 1: N350
- Dangote to Julius Berger using Truck 2: N300
- Elephant to Intra using Truck 1: N400
- Elephant to Intra using Truck 2: N300
- Elephant to SCC using Truck 1: N400
- Elephant to SCC using Truck 2: N350
- Elephant to Julius Berger using Truck 1: N350
- Elephant to Julius Berger using Truck 2: N350

**Vehicle Capacities:**

- Capacity of Truck 1: 600 units
- Capacity of Truck 2: 700 units

**Decision Variables:**

- Let $(x_{s,c,v})$ represent the quantity of goods to be shipped from supplier $s$ to consumer $c$ using vehicle type $v$.

**Objective Function:**

Minimize the total transportation cost:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  Minimize $\sum_{s}\sum_{c}\sum_{v} x_{s,c,v} \times transportation\_cost_{s,c,v}$

**Constraints:**

1. Supply constraints: For each supplier, the shipped quantity cannot exceed the supply:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $\sum_{c}\sum_{v} x_{s,c,v} \le supply_{s}$ for each supplier $s$ 

2. Demand constraints: For each consumer, the received quantity must meet the demand:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  $\sum_{c}\sum_{v} x_{s,c,v} = demand_{s}$ for each consumer $c$ 

3. Vehicle capacity constraints: For each supplier and vehicle type, the shipped quantity must not exceed the vehicle's capacity:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  $\sum_{c}\sum_{v} x_{s,c,v} \le Vehicle\_Capacity_{v}$ for each supplier $s$  and each vehicle type $v$


In [9]:
from pulp import *

# Create a linear programming problem instance
problem = pulp.LpProblem("Transportation_Problem", LpMinimize)

# Data
suppliers = ['Dangote', 'Elephant']
consumers = ['Intra', 'SCC', 'Julius Berger']
vehicle_types = ['Truck 1', 'Truck 2']

# Supply and demand
supply = {'Dangote': 700, 'Elephant': 800}
demand = {'Intra': 550, 'SCC': 500, 'Julius Berger': 450}

# Transportation cost per unit
transportation_cost = {
    ('Dangote', 'Intra', 'Truck 1'): 400,
    ('Dangote', 'Intra', 'Truck 2'): 350,
    ('Dangote', 'SCC', 'Truck 1'): 350,
    ('Dangote', 'SCC', 'Truck 2'): 350,
    ('Dangote', 'Julius Berger', 'Truck 1'): 350,
    ('Dangote', 'Julius Berger', 'Truck 2'): 300,
    ('Elephant', 'Intra', 'Truck 1'): 400,
    ('Elephant', 'Intra', 'Truck 2'): 300,
    ('Elephant', 'SCC', 'Truck 1'): 400,
    ('Elephant', 'SCC', 'Truck 2'): 350,
    ('Elephant', 'Julius Berger', 'Truck 1'): 350,
    ('Elephant', 'Julius Berger', 'Truck 2'): 350,
}

# Vehicle capacities
v_capacity = {'Truck 1': 600, 'Truck 2': 700}
# Decision variables
variables = LpVariable.dicts("Shipment", 
                                  ((s, c, v) for s in suppliers for c in consumers for v in vehicle_types),
                                  lowBound=0, cat="Integer")

# Objective function
problem += lpSum(variables[s, c, v] * transportation_cost[s, c, v]
                      for s in suppliers for c in consumers for v in vehicle_types)

# Supply constraints
for s in suppliers:
    problem += lpSum(variables[s, c, v] for c in consumers for v in vehicle_types) <= supply[s]

# Demand constraints
for c in consumers:
    problem += lpSum(variables[s, c, v] for s in suppliers for v in vehicle_types) == demand[c]

# Vehicle capacity constraints
for v in vehicle_types:
    for s in suppliers:
        problem += lpSum(variables[s, c, v] for c in consumers) <= v_capacity[v]


print(problem)
# Solve the problem
problem.solve()

print(f"Problem Status: {LpStatus[problem.status]}")

# Print results
print("\nOptimal Solution:")
for s in suppliers:
    for c in consumers:
        for v in vehicle_types:
            if variables[s, c, v].value() >0:
                print(f"Ship {variables[s, c, v].value()} units of cements from {s} to {c} using {v} cost N{transportation_cost[s,c,v]} per unit")

print(f"\nTotal Transportation Cost: N{value(problem.objective)}")



Transportation_Problem:
MINIMIZE
400*Shipment_('Dangote',_'Intra',_'Truck_1') + 350*Shipment_('Dangote',_'Intra',_'Truck_2') + 350*Shipment_('Dangote',_'Julius_Berger',_'Truck_1') + 300*Shipment_('Dangote',_'Julius_Berger',_'Truck_2') + 350*Shipment_('Dangote',_'SCC',_'Truck_1') + 350*Shipment_('Dangote',_'SCC',_'Truck_2') + 400*Shipment_('Elephant',_'Intra',_'Truck_1') + 300*Shipment_('Elephant',_'Intra',_'Truck_2') + 350*Shipment_('Elephant',_'Julius_Berger',_'Truck_1') + 350*Shipment_('Elephant',_'Julius_Berger',_'Truck_2') + 400*Shipment_('Elephant',_'SCC',_'Truck_1') + 350*Shipment_('Elephant',_'SCC',_'Truck_2') + 0
SUBJECT TO
_C1: Shipment_('Dangote',_'Intra',_'Truck_1')
 + Shipment_('Dangote',_'Intra',_'Truck_2')
 + Shipment_('Dangote',_'Julius_Berger',_'Truck_1')
 + Shipment_('Dangote',_'Julius_Berger',_'Truck_2')
 + Shipment_('Dangote',_'SCC',_'Truck_1')
 + Shipment_('Dangote',_'SCC',_'Truck_2') <= 700

_C2: Shipment_('Elephant',_'Intra',_'Truck_1')
 + Shipment_('Elephant',_'I

## Expalanation of Solution
**Total Transportation Cost:**
The total transportation cost is the sum of the costs associated with shipping goods from each supplier to each consumer using different vehicle capacity. The objective function is designed to minimize this total cost.

As Obtained, the calculated total transportation cost is N2450, it means that the chosen shipping plan minimizes the cost of transporting the goods from suppliers to consumers, taking into account the transportation cost per unit for each route.

**Optimality:**
The solution is considered optimal when the transportation cost cannot be further reduced while satisfying all constraints. The optimization algorithm ensures that the chosen shipping plan is the most cost-efficient considering the provided data and constraints.